# Step 1: Implementing CNN

## Description


I utilized the CIFAR-10 dataset and a tutorial for CNNs for this dataset (https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) in order to accomplish this task. Below are the different experimentats I performed both across hyperparameters and network architectures.

1.   Learning Rates

* lr=0.1 -> 10 %
* lr=0.01 -> 28 %
* **lr=0.001 -> 39 %**

2.   Epochs

* epochs: 2 -> 39 %
* epochs: 3 -> 42 %
* epochs: 4 ->  57 % %
* **epochs: 5 -> 59 %**

3.   Fully-connected Layers

* I attempted two architectures for fully-connected layers, one as follows:

* self.fc1 = nn.Linear(16 * 5 * 5, 200)

* self.fc2 = nn.Linear(200, 120)

* self.fc3 = nn.Linear(120, 84)

* self.fc4 = nn.Linear(84, 10)

* accuracy : 54 %

* next attempt:

* .fc1 = nn.Linear(16 * 5 * 5, 120)
* self.fc2 = nn.Linear(120, 84)
* self.fc3 = nn.Linear(84, 10)
* **accuracy : 59 %**


---


Ultimately, I concluded that the optimal combination for performance was:
* 5 epochs
* learning rate of 0.001
* 3 fully-connected layers

## Code

In [ ]:
# download / normalize data

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# define CNN

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
import torch.optim as optim
import tqdm

# train network

# STEP 1: experimentations on different hyperparameters:
epochs = 5
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

[1,  2000] loss: 2.251
[1,  4000] loss: 1.947
[1,  6000] loss: 1.730
[1,  8000] loss: 1.625
[1, 10000] loss: 1.545
[1, 12000] loss: 1.477
[2,  2000] loss: 1.422
[2,  4000] loss: 1.372
[2,  6000] loss: 1.367
[2,  8000] loss: 1.332
[2, 10000] loss: 1.298
[2, 12000] loss: 1.274
[3,  2000] loss: 1.217
[3,  4000] loss: 1.224
[3,  6000] loss: 1.210
[3,  8000] loss: 1.182
[3, 10000] loss: 1.205
[3, 12000] loss: 1.182
[4,  2000] loss: 1.102
[4,  4000] loss: 1.110
[4,  6000] loss: 1.144
[4,  8000] loss: 1.091
[4, 10000] loss: 1.108
[4, 12000] loss: 1.128
[5,  2000] loss: 1.022
[5,  4000] loss: 1.048
[5,  6000] loss: 1.041
[5,  8000] loss: 1.087
[5, 10000] loss: 1.042
[5, 12000] loss: 1.062
Finished Training


In [ ]:
# reload saved network
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
#test on whole dataset
import matplotlib.pyplot as plt
import numpy as np

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 59 %


# Step 2: Considering MEC

## Description

In order the calulate the minimum MEC needed for a CNN of this dataset, we can use the following formulae:

> batch_size * channel * width * height

> output = input / compression ratio

Applying this to our specific case, we get:

> (4 * 3 * 32 * 32) / (192/25) = **1600 minimum MEC**

So, 1600 is the desired MEC for the function. Below, I have calculated the MEC for my current implementation based on similar calculations from part 8.1.

> (400 * 120) + min(120 * 84, 84) + min(84 * 10, 10) = **48,094 current MEC**

The current MEC of my implementation is much larger than it should be. This indicates that I can find ways to reduce or readjust the architecture in my CNN to reduce the MEC to a value closer to 1600, and this should maintain a similar accuracy to the 59% from before. I attempt exactly this by making my new CNN only contain one single fully-connected layer instead of 3. The new architecture looks as follows:

> self.fc1 = nn.Linear(16 * 5 * 5, 10)

Essentially, the layer reduces from 400 to 10. My new MEC would then be:

> 400 * 10 + min(400 * 10, 10) = **4,010 new MEC**

While this new MEC is still quite a bit larger than 1600, it is much closer than the previous solution, and it reduces a significant amount of memory usage by only employing one linear layer.

After training on the same hyperparameters as the previous model, this model achieved a **57% accuracy**, which is quite similar to the original while reducing memory usage by a large amount.

## Code

In [ ]:
# removing all but 1 fully-connected layer

import torch.nn as nn
import torch.nn.functional as F

class New_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #reducing to one linear layer reducing from 400 to 10
        self.fc1 = nn.Linear(16 * 5 * 5, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

net = New_Net()

In [ ]:
import torch.optim as optim

#running on 5 epochs and learning rate 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

PATH = './cifar_net_new.pth'
torch.save(net.state_dict(), PATH)

[1,  2000] loss: 1.983
[1,  4000] loss: 1.698
[1,  6000] loss: 1.571
[1,  8000] loss: 1.506
[1, 10000] loss: 1.454
[1, 12000] loss: 1.424
[2,  2000] loss: 1.360
[2,  4000] loss: 1.368
[2,  6000] loss: 1.321
[2,  8000] loss: 1.323
[2, 10000] loss: 1.308
[2, 12000] loss: 1.319
[3,  2000] loss: 1.227
[3,  4000] loss: 1.267
[3,  6000] loss: 1.232
[3,  8000] loss: 1.240
[3, 10000] loss: 1.232
[3, 12000] loss: 1.231
[4,  2000] loss: 1.159
[4,  4000] loss: 1.208
[4,  6000] loss: 1.171
[4,  8000] loss: 1.182
[4, 10000] loss: 1.198
[4, 12000] loss: 1.181
[5,  2000] loss: 1.168
[5,  4000] loss: 1.144
[5,  6000] loss: 1.160
[5,  8000] loss: 1.169
[5, 10000] loss: 1.146
[5, 12000] loss: 1.156
Finished Training


In [ ]:
new_net = New_Net()
new_net.load_state_dict(torch.load(PATH))

#outputs = new_net(images)

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = new_net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')